<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_GRU%EA%B8%B0%EB%B0%98_%EC%83%9D%EC%84%B1AI_%EB%AA%A8%EB%8D%B8%EB%A7%81_ver4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kiwipiepy

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 48.7 MB/s eta 0:00:00


In [69]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

from sklearn.utils import resample

import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 데이터 불러오기

### 전체 데이터셋

In [14]:
pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct.csv").head(3)

,review,reply,star,star_t,star_q,star_d,food,store,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,5,5.0,5.0,5.0,직화 삼겹옛날도시락,['1988응답하라추억의옛날도시락 방이점'],1
1,기름 있는 부분이 생각보다 많아요 참고하세요,소중한 의견 남겨 주셔서 감사합니다 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 노...,5,5.0,5.0,5.0,직화 삼겹옛날도시락 볶음김치,['1988응답하라추억의옛날도시락 방이점'],0
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,5.0,5.0,5.0,직화 삼겹옛날도시락 우삼겹청국장찌개 대접밥,['1988응답하라추억의옛날도시락 방이점'],1


In [7]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct.csv")
cols = ['review','reply','sentiment']
train = train[cols]
train.isnull().sum()

review        0
reply        48
sentiment     0
dtype: int64

In [8]:
idx_null = train[train['reply'].isnull()].index
train = train.drop(idx_null, axis=0)
train.isnull().sum()

review       0
reply        0
sentiment    0
dtype: int64

In [9]:
train['sentiment'].value_counts().sort_index() # 긍부정 분포 확인

0     1813
1    35279
Name: sentiment, dtype: int64

In [10]:
print(len((train[train['sentiment'] == 0])))
len(train[train['sentiment'] == 0])/len(train[train['sentiment'] == 1])

1813


0.051390345531335924

### 오버샘플링(긍정->부정 데이터)

In [22]:
pd.read_csv(f"{DATA_PATH}train_gpt_4400.csv").head(3)

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0


In [23]:
train_gpt = pd.read_csv(f"{DATA_PATH}train_gpt_4400.csv")
train_gpt.isnull().sum()

gpt_review    0
gpt_reply     0
sentiment     0
dtype: int64

In [24]:
# 컬럼명 통일
rename_col = {'gpt_review': 'review', 'gpt_reply': 'reply'}
train_gpt.rename(columns=rename_col, inplace=True)
train_gpt

,review,reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
4396,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,0
4397,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,0
4398,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,0
4399,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,0


In [25]:
# 합치기
train = pd.concat([train, train_gpt], axis=0).reset_index(drop=True)
train

,review,reply,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,1
1,기름 있는 부분이 생각보다 많아요 참고하세요,소중한 의견 남겨 주셔서 감사합니다 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 노...,0
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1
3,,저희 매장과 함께 편안한 시간 되셨을까요 1 고객을 생각하는 마음 2 더 나은 맛과...,1
4,냉면을 따로 시키고 미니 냉면을 합친 것인데 양이 좀 적어 보이고 김치 정식도 양이...,주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요 고객님의 행복한 시간에 저희...,1
...,...,...,...
41488,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,0
41489,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,0
41490,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,0
41491,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,0


In [26]:
train['review'] = train['review'].astype(str)
train['reply'] = train['reply'].astype(str)

### 여기까지 저장하기

In [27]:
# train.to_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct_gpt.csv", index=False)

# 여기까지 완료된 데이터 불러오기

In [28]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct_gpt.csv")
train.head(3)

,review,reply,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,1
1,기름 있는 부분이 생각보다 많아요 참고하세요,소중한 의견 남겨 주셔서 감사합니다 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 노...,0
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1


In [29]:
train.isnull().sum()

review       0
reply        0
sentiment    0
dtype: int64

## 추가 전처리

### 개별 삭제(추론 결과보고 계속 추가)

In [ ]:
# print(train[train['reply'].str.contains("뜸들이다는 북서울꿈의숲")].index)

In [ ]:
# remove_keywords = ['뜸들이다는 북서울꿈의숲']
# for keyword in remove_keywords:
#     train['reply'] = train['reply'].str.replace(keyword, '')

# train['reply'][6822]

### 단문 데이터 제거
- 데이터 정제 과정에서 무의미해진 데이터는 학습데이터로 사용불가
    - 리뷰 : 공백만 생겼거나, 리뷰 이벤트를 위한 단답형 제거
    - 답변 : 16자 이하 답변 제거

In [35]:
len(train['review'][3]) , len(train['reply'][642])

(1, 3)

In [48]:
train['review'][3] , train['reply'][642]

(' ', '이렇게')

In [51]:
train[train['review'].str.len() <= 3] # 리뷰

,review,reply,sentiment
3,,저희 매장과 함께 편안한 시간 되셨을까요 1 고객을 생각하는 마음 2 더 나은 맛과...,1
147,,안녕하세요 든든한 한 끼 되셨을까요 가족에게 드리는 마음으로 나가고 있는데 입맛에 ...,1
548,,안녕하세요 입에는 맞으셨나요 주문 주셔서 감사합니다,1
550,,사이드만 잔뜩 시키고 1,1
740,굿,안녕하세요 맛있게 드셔주셔서 감사합니다 저희 드시면서 좋은 하루 되셨길 바랍니다 고...,1
...,...,...,...
36397,짱,리뷰도 짱짱 감사합니다 나중에 생각 마실 때 또 찾아쥬세용,1
36694,굿,고객님의 최고의 만족을 위해 노력하는 고객님의 한 끼 식사가 든든하게 채워질 수 있...,1
36739,,안녕하세요 고객님의 한 끼 식사를 위해 항상 노력하는 전 직원이 항상 고객님의 만족...,1
36742,,안녕하세요 고객님의 한 끼 식사를 위해 항상 노력하는 전 직원이 항상 고객님의 만족...,1


In [47]:
train[(train['reply'].str.len() > 15) & (train['reply'].str.len() <= 17)] # 답변

,review,reply,sentiment
556,맛있어요 감사 허무니 당,네넵 감사 감사 감사 허무니 당,1
561,배달도 예상시간보다 빨리 왔고 치즈도 엄청 많이 왔어요 진짜 맛있습니다,양도 맛도 항상 진심 감사합니다,1
651,오오 떡볶이도 진짜 맛있고 튀김도 맛있네요,감사합니다 자주 이용해 주세요,1
685,아쉽게 사진을 찍진 못했지만 맛있게 잘 먹었어요,맛있게 드셨다니 너무 감사합니다,1
688,양도 푸짐하고 맛도 괜찮네요,감사해요 또 만났으면 좋겠습니다,1
...,...,...,...
40790,지금껏 먹은 닭볶음탕중에 최고입니다~ 정말 맛있네요 자주 시켜먹을께요 맛이 별로였...,"사장님의 답변이 없습니다 ,",0
40791,"몇번째 주문인지 다른 배달 어플포함 여튼 엄청 맛있어욬 맛이 별로였어요, 음식이 ...","사장님의 답변이 없습니다 ,",0
40792,여기는 무조건 때되면 시켜먹어야되요 안그러면 금단 증상옴 계속 생각남 맛이 별로였...,"사장님의 답변이 없습니다 ,",0
40793,"배달도빠르고 양도넉넉하구 그래서 맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹...","사장님의 답변이 없습니다 ,",0


In [53]:
len(train[train['review'].str.len() <= 3]) , len(train[train['reply'].str.len() <= 17])

(322, 1024)

In [56]:
train = train[(train['review'].str.len() > 3) & (train['reply'].str.len() > 17)].reset_index(drop=True)
train

,review,reply,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,1
1,기름 있는 부분이 생각보다 많아요 참고하세요,소중한 의견 남겨 주셔서 감사합니다 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 노...,0
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1
3,냉면을 따로 시키고 미니 냉면을 합친 것인데 양이 좀 적어 보이고 김치 정식도 양이...,주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요 고객님의 행복한 시간에 저희...,1
4,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께해 주셔서 영광 다음에도 고객님의 즐거운 순간을 함...,1
...,...,...,...
40164,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,0
40165,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,0
40166,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,0
40167,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,0


## 긍부정 데이터 비율 선택

In [57]:
train['sentiment'].value_counts().sort_index() # 긍부정 분포 확인

0     6108
1    34061
Name: sentiment, dtype: int64

In [58]:
train_neg = train[train['sentiment'] == 0]

# 긍정 데이터 중 중복된 답변제거
train_pos = train[train['sentiment'] == 1].drop_duplicates(subset='reply', keep=False)
train_pos['sentiment'].value_counts().sort_index()

1    15433
Name: sentiment, dtype: int64

In [59]:
length = len(train_neg)
length

6108

In [60]:
# 긍정 데이터를 랜덤으로 추출하기 위해 섞기
shuffled_train_pos = train_pos.sample(frac=1, random_state=42)
# train_pos = shuffled_train_pos.iloc[:length].reset_index(drop=True) # 1:1 비율
train_pos = shuffled_train_pos.iloc[:length*2].reset_index(drop=True) # 2:1 비율 만들기

# 데이터 합치기
train = pd.concat([train_neg, train_pos], ignore_index=True)
train['sentiment'].value_counts().sort_index()

0     6108
1    12216
Name: sentiment, dtype: int64

In [61]:
# 다시 섞기
train = train.sample(frac=1, random_state=42).reset_index(drop=True)
train

,review,reply,sentiment
0,처음 포장해서 먹었는데 포장해도 존마탱 너무 맛있어요 혹시 다음에 기억하시면 시래깃...,아이고 첫 주문이 마음에 드신 것 같아 다행이네요 하실 일 있으시면 요청사항에 나 ...,1
1,배고파서 사진을 막 찍었네요 맛도 최고 가격 최고 서비스도 최고 배달도 빨리 와서 ...,맛있게 드셨다니 너무나 감사힙니당 앞으로도 늘 변함없이 더욱더 푸짐한 양과 최고의 ...,1
2,양도 많고 바삭하니 맛있어요,고객님 양도 많고 바삭하게 맛있게 드시고 사진과 별 리뷰까지 감사드립니다 양이 부족...,1
3,생일 케이크가 필요하던 참에 후기도 너무 좋고 맛있을 것 같아 피한 딸기 몽땅 케이...,어머나 특별한 날 주문 주시기에 아쉬움이 있으면 너무 속상한데 그럼에도 불구하고 좋...,0
4,잘 먹었습니다 맛있어요,맛있게 드셔주시고 남겨주신 소중한 리뷰와 별 5개 WOW 감사합니다 고객님 덕분에 ...,1
...,...,...,...
18319,역시나 소낙비 너무 맛있어요 국수나무는 진짜 알밥이랑 소낙비 최고 아쿠아 돈가스는 ...,맛있게 드셔 주셔서 감사합니다 아쿠아 많이 새콤하셨으면 로제 추천드려요 항상 만족하...,1
18320,역시 전통은 전통이네요 또 시켜 먹을게요,안녕하세요 알아주시니 너무 감사드립니다 앞으로 맛나게 찾아 뵙겠습니다 건강하세요,1
18321,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시...","죄송합니다, 앞으로는 촉촉한 음식을 제공하기 위해 최선을 다하겠습니다. ,",0
18322,리뷰 좀 보니까 배달이 좀 늦은 편이라고 하긴 했는데 정말 좀 늦어서 아쉽긴 했어요...,소중한 리뷰 정말 감사드립니다 좀 식으셨군요 정말 죄송합니다 따뜻한 드실 수 있게 ...,0


In [62]:
review, reply = train["review"] , train["reply"]
len(review) , len(reply)

(18324, 18324)

# 한국어 형태소 분석

In [63]:
review[:5]

0    처음 포장해서 먹었는데 포장해도 존마탱 너무 맛있어요 혹시 다음에 기억하시면 시래깃...
1    배고파서 사진을 막 찍었네요 맛도 최고 가격 최고 서비스도 최고 배달도 빨리 와서 ...
2                                      양도 많고 바삭하니 맛있어요
3    생일 케이크가 필요하던 참에 후기도 너무 좋고 맛있을 것 같아 피한 딸기 몽땅 케이...
4                                         잘 먹었습니다 맛있어요
Name: review, dtype: object

# 단어 사전 만들기

In [64]:
PAD = 0
SOS = 1
EOS = 2

class WordVocab():
    def __init__(self):
        self.word2index = {'<PAD>': PAD,
                           '<SOS>': SOS,
                           '<EOS>': EOS,}
        self.word2count = {}
        self.index2word = {PAD : '<PAD>',
                           SOS : '<SOS>',
                           EOS : '<EOS>'}
        self.n_words = 3  # PAD, SOS, EOS 포함

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [65]:
print(f'원문: {review[0]}')
lang = WordVocab()
lang.add_sentence(review[0])
print('단어사전: ', lang.word2index)

원문: 처음 포장해서 먹었는데 포장해도 존마탱 너무 맛있어요 혹시 다음에 기억하시면 시래깃국이나 콩나물이면 안 주셔도 됩니다 
단어사전:  {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '처음': 3, '포장해서': 4, '먹었는데': 5, '포장해도': 6, '존마탱': 7, '너무': 8, '맛있어요': 9, '혹시': 10, '다음에': 11, '기억하시면': 12, '시래깃국이나': 13, '콩나물이면': 14, '안': 15, '주셔도': 16, '됩니다': 17, '': 18}


# 데이터셋 만들기

In [66]:
from konlpy.tag import Okt

In [67]:
class TextDataset(Dataset):
    def __init__(self, df, min_length=3, max_length=64):
        super().__init__()

        # TOKEN 정의
        self.PAD = 0
        self.SOS = 1
        self.EOS = 2

        self.tagger = Okt() # 트위터 기반 형태소 분석기
        self.max_length = max_length

        # 형태소 처리와 단어사전 정의가 된 토큰을 담은 리스트
        src_clean = []
        tgt_clean = []

        # 단어 사전 생성
        wordvocab = WordVocab()

        for _, row in df.iterrows():
            # df 에서 'src'와 'tgt' 정의
            src = row['review']
            tgt = row['reply']

            # 형태소 처리
            src = self.clean_text(src)
            tgt = self.clean_text(tgt)

            if len(src.split()) > min_length and len(tgt.split()) > min_length:
                # 최소 길이를 넘어가는 문장의 단어만 추가
                wordvocab.add_sentence(src)
                wordvocab.add_sentence(tgt)
                src_clean.append(src)
                tgt_clean.append(tgt)

        self.srcs = src_clean
        self.tgts = tgt_clean
        self.wordvocab = wordvocab

    def clean_text(self, sentence):
        # 형태소 처리
        sentence = self.tagger.morphs(sentence)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()
        return sentence

    def texts_to_sequences(self, sentence):
        # 문장 -> 시퀀스로 변환
        return [self.wordvocab.word2index[w] for w in sentence.split()]

    def pad_sequence(self, sentence_tokens):
        # 문장의 맨 끝 토큰은 제거
        sentence_tokens = sentence_tokens[:(self.max_length-1)]
        token_length = len(sentence_tokens)

        # 문장의 맨 끝부분에 <EOS> 토큰 추가
        sentence_tokens.append(self.EOS)

        for i in range(token_length, (self.max_length-1)):
            # 나머지 빈 곳은 <PAD> 토큰으로 채우기
            sentence_tokens.append(self.PAD)
        return sentence_tokens

    def __len__(self):
        return len(self.srcs)

    def __getitem__(self, idx):
        inputs = self.srcs[idx]
        inputs_sequences = self.texts_to_sequences(inputs) # 시퀀스로 바꾸고
        inputs_padded = self.pad_sequence(inputs_sequences) # pad 토큰 채우기

        outputs = self.tgts[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)

        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)

In [68]:
MAX_LENGTH = 64
dataset = TextDataset(train, min_length=3, max_length=MAX_LENGTH)

## 데이터셋 분할(테스트 데이터 생성)

In [70]:
x, y = dataset[0]

In [71]:
print(f'x shape: {x.shape}')
print(x)

x shape: torch.Size([64])
tensor([ 3,  4,  5,  6,  4,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [72]:
print(f'y shape: {y.shape}')
print(y)

y shape: torch.Size([64])
tensor([27, 28, 29, 22, 30, 14, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 14,
        42, 43, 44, 45,  7, 29, 22, 12, 42, 46, 47, 48, 49, 50, 51, 36, 52, 53,
        54, 55, 56, 57, 10, 58, 59, 52, 60, 28, 29, 22, 61, 29, 62, 63, 64, 65,
        66, 67, 55, 68, 69,  2,  0,  0,  0,  0])


In [73]:
from torch.utils.data import SubsetRandomSampler, random_split

train_size = int(len(dataset) * 0.8) # 80%의 데이터를 train에 할당
test_size = len(dataset) - train_size # 나머지 20% 데이터를 test에 할당
print(train_size, test_size)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

13067 3267


# 데이터로더 만들기

In [74]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [75]:
batch = next(iter(train_loader))
batch

[tensor([[ 145,  953,  695,  ...,    0,    0,    0],
         [  52,   60,  713,  ...,    0,    0,    0],
         [  28,   29, 3875,  ...,    0,    0,    0],
         ...,
         [  98,   22,   10,  ...,    0,    0,    0],
         [ 953,  695,   14,  ...,    0,    0,    0],
         [1882,   22,   10,  ...,    0,    0,    0]]),
 tensor([[2308,   36,   65,  ...,  107,   58,    2],
         [2657, 7616, 1345,  ...,  534, 2052,    2],
         [  28,   29, 6715,  ...,    0,    0,    0],
         ...,
         [  98,   22, 4481,  ...,    0,    0,    0],
         [ 702,  118, 1943,  ...,    0,    0,    0],
         [1882,   22,  956,  ...,    0,    0,    0]])]

In [76]:
# 1개의 배치 데이터를 추출
x, y = next(iter(train_loader))

# shape: (batch_size, sequence_length)
x.shape, y.shape

(torch.Size([16, 64]), torch.Size([16, 64]))

# Encoder 만들기

In [77]:
class Encoder(nn.Module):
    def __init__(self, num_embeddings, hidden_size, embedding_dim, num_layers):
        super().__init__()
        self.num_embeddings = num_embeddings # vocab size

        # 임베딩 레이어 정의 (number of vocabs, embedding dimension)
        self.emb_layer = nn.Embedding(num_embeddings, embedding_dim)

        # GRU (embedding dimension)
        self.rnn_layer = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

    def forward(self, x):
        x = self.emb_layer(x).permute(1, 0, 2)
        outputs, hn = self.rnn_layer(x)
        return outputs, hn

# Decoder 만들기

In [78]:
class Decoder(nn.Module):
    def __init__(self, num_embeddings, hidden_size, embedding_dim, num_layers=1, dropout=0.2):
        super().__init__()
        self.num_embeddings = num_embeddings
        self.emb_layer = nn.Embedding(num_embeddings, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.rnn_layer = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          batch_first=False) # (1, batch_size) 로 변환을 쉽게 하기 위함

        # 최종 출력은 단어사전의 개수(num_embeddings)
        self.fc_layer = nn.Linear(hidden_size, num_embeddings)

    def forward(self, x, hidden_state):
        x = x.unsqueeze(0) # (1, batch_size) 로 변환
        x = F.relu(self.emb_layer(x))
        x = self.dropout(x)
        outputs, hn = self.rnn_layer(x, hidden_state)
        outputs = self.fc_layer(outputs.squeeze(0)) # (sequence_length, batch_size, hidden_size(32))
        return outputs, hn

# Seq2Seq 모델 만들기

In [79]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        # inputs : (batch_size, sequence_length)
        # outputs: (batch_size, sequence_length)

        batch_size, output_length = outputs.shape
        output_num_vocabs = self.decoder.num_embeddings

        # 리턴할 예측된 outputs를 저장할 임시 변수
        # (sequence_length, batch_size, num_vocabs)
        predicted_outputs = torch.zeros(output_length, batch_size, output_num_vocabs).to(self.device)

        # 인코더에 입력 데이터 주입, encoder_output은 버리고 hidden_state 만 살림(Attention 안하니까 필요없음)
        # 여기서 hidden_state가 디코더에 주입할 context vector 가 됨
        # (number of layers(1), batch_size, hidden_size)
        _, decoder_hn = self.encoder(inputs)

        # (batch_size) shape의 SOS TOKEN으로 채워진 디코더 입력 생성
        decoder_input = torch.full((batch_size,), SOS, device=self.device)

        # 반복문으로 출력 단어를 생성
        # 0번째는 SOS TOKEN이 위치하므로, 1번째 인덱스부터 순회
        for t in range(0, output_length):
            # decoder_input : 디코더 입력 (batch_size) 형태의 SOS TOKEN로 채워진 입력
            # decoder_output: (batch_size, num_vocabs)
            # decoder_hidden: (number of layers(1), batch_size, hidden_size), context vector와 동일 shape
            decoder_output, decoder_hn = self.decoder(decoder_input, decoder_hn)

            # t번째 단어에 디코더의 output 저장
            predicted_outputs[t] = decoder_output

            # teacher forcing 적용 여부 확률로 결정
            # 정답치를 다음 RNN Cell의 입력으로 넣어주는 경우. 수렴속도가 빠를 수 있으나, 불안정할 수 있음
            teacher_force = random.random() < teacher_forcing_ratio

            # top1 단어 토큰 예측
            top1 = decoder_output.argmax(1)

            # teacher forcing 인 경우 ground truth 값을, 그렇지 않은 경우, 예측 값을 다음 input으로 지정
            decoder_input = outputs[:, t] if teacher_force else top1

        return predicted_outputs.permute(1, 0, 2) # (batch_size, sequence_length, num_vocabs)로 변경

In [80]:
# Encoder 정의
encoder = Encoder(num_embeddings=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Decoder 정의
decoder = Decoder(num_embeddings=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Seq2Seq 정의
seq2seq = Seq2Seq(encoder, decoder, 'cpu')

In [81]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)
# (batch_size, sequence_length), (batch_size, sequence_length)

torch.Size([16, 64]) torch.Size([16, 64])


In [82]:
output = seq2seq(x, y)
print(output.shape)
# (batch_size, sequence_length, num_vocabs)

torch.Size([16, 64, 24623])


## 모델 확인

In [83]:
NUM_VOCABS = dataset.wordvocab.n_words
HIDDEN_SIZE = 512
EMBEDDIMG_DIM = 256

print(f'num_vocabs: {NUM_VOCABS}\n======================')

# Encoder 정의
encoder = Encoder(num_embeddings=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)
# Decoder 정의
decoder = Decoder(num_embeddings=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)

# Seq2Seq 생성
# encoder, decoder를 device 모두 지정
model = Seq2Seq(encoder.to(device), decoder.to(device), device)
print(model)

num_vocabs: 24623
Seq2Seq(
  (encoder): Encoder(
    (emb_layer): Embedding(24623, 256)
    (rnn_layer): GRU(256, 512)
  )
  (decoder): Decoder(
    (emb_layer): Embedding(24623, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn_layer): GRU(256, 512)
    (fc_layer): Linear(in_features=512, out_features=24623, bias=True)
  )
)


# 하이퍼파라미터 정의

In [84]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0

        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta


    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')

        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')


        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

In [85]:
LR = 1e-3
optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
es = EarlyStopping(patience=5, delta=0.001, mode='min', verbose=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

# 학습 loop 만들기

In [86]:
def train(model, data_loader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0

    for x, y in data_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        # output: (batch_size, sequence_length, num_vocabs)
        output = model(x, y)
        output_dim = output.size(2)

        # 1번 index 부터 슬라이싱한 이유는 0번 index가 SOS TOKEN 이기 때문
        # (batch_size*sequence_length, num_vocabs) 로 변경
        output = output.reshape(-1, output_dim)

        # (batch_size*sequence_length) 로 변경
        y = y.view(-1)

        # Loss 계산
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)

    return running_loss / len(data_loader)

In [87]:
def evaluate(model, data_loader, loss_fn, device):
    model.eval()

    eval_loss = 0

    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            output = model(x, y)
            output_dim = output.size(2)
            output = output.reshape(-1, output_dim)
            y = y.view(-1)

            # Loss 계산
            loss = loss_fn(output, y)

            eval_loss += loss.item() * x.size(0)

    return eval_loss / len(data_loader)

# 추론 함수

In [88]:
def sequence_to_sentence(sequences, index2word):
    outputs = []
    for p in sequences:

        word = index2word[p]
        if p not in [SOS, EOS, PAD]:
            outputs.append(word)
        if word == EOS:
            break
    return ' '.join(outputs)

# 테스트 loop 만들기
- 챗봇 함수화

In [ ]:
# # 전체 예측값을 데이터프레임 형식에 담기
# def random_evaluation(model, dataset, index2word, device, n=10):

#     n_samples = len(dataset)
#     indices = list(range(n_samples))
#     np.random.shuffle(indices)      # Shuffle
#     sampled_indices = indices[:n]   # Sampling N indices

#     # 샘플링한 데이터를 기반으로 DataLoader 생성
#     sampler = SubsetRandomSampler(sampled_indices)
#     sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

#     model.eval()
#     with torch.no_grad():
#         for x, y in sampled_dataloader:
#             x, y = x.to(device), y.to(device)
#             output = model(x, y, teacher_forcing_ratio=0)
#             # output: (number of samples, sequence_length, num_vocabs)

#             preds = output.detach().cpu().numpy()
#             x = x.detach().cpu().numpy()
#             y = y.detach().cpu().numpy()

#             data = {'test_review': [], 'chatbot_reply': []}
#             for i in range(n_samples):
#                 x_result = sequence_to_sentence(x[i], index2word)
#                 y_result = sequence_to_sentence(y[i], index2word)
#                 pred_result = sequence_to_sentence(preds[i].argmax(1), index2word)
#                 pred_result_correct = correct_spell(pred_result)
#                 data['test_review'].append(x_result)
#                 data['chatbot_reply'].append(pred_result_correct)

#     df = pd.DataFrame(data)
#     print(df)

In [89]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(text, generated_text):
    y = [text]  # 실제 데이터 y
    X = generated_text  # 생성된 텍스트 X

    # BLEU 점수 계산
    bleu_score = sentence_bleu(y, X)
    return bleu_score

In [90]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=128, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=2,
                          no_repeat_ngram_size=4,
                          top_k=5,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

In [91]:
# 10개만 보기
def random_evaluation_10(model, dataset, index2word, device, n=10):

    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices

    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)

            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()

            for i in range(n):
                result_review = sequence_to_sentence(x[i], index2word)
                result_reply = sequence_to_sentence(y[i], index2word)
                result_pred = sequence_to_sentence(preds[i].argmax(1), index2word)
                print(f'리뷰    : {result_review}')
                print(f'답변    : {result_reply}')
                print(f'예측답변: {paraphrase_text(result_pred)}') # paraphrased text 반환
                print(f'BLEU점수: {calculate_bleu(result_reply, result_pred)}')
                print('==='*10)

# 학습하기

In [ ]:
# yogiyo_paraphrase
NUM_EPOCHS = 30
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0908_gpt.pt'

best_loss = np.inf

for epoch in tqdm(range(NUM_EPOCHS), total=NUM_EPOCHS):
    loss = train(model, train_loader, optimizer, loss_fn, device)

    val_loss = evaluate(model, test_loader, loss_fn, device)

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), STATEDICT_PATH)

    if epoch % 5 == 0:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')

    # Early Stop
    es(loss)
    if es.early_stop:
        break

    # Scheduler
    scheduler.step(val_loss)

model.load_state_dict(torch.load(STATEDICT_PATH))
torch.save(model.state_dict(), f'/content/drive/MyDrive/project_final/data/models/yogiyo_0908_gpt{best_loss:.4f}.pt')

  0%|          | 0/30 [00:00<?, ?it/s]

epoch: 1, loss: 51.6629, val_loss: 44.9030
[EarlyStopping] (Update) Best Score: 51.66291
[EarlyStopping] (Update) Best Score: 43.04779
[EarlyStopping] (Update) Best Score: 39.55389


# 결과 확인하기

In [ ]:
# 0908 데이터 + gpt + 긍부정 비율 2:1 + 에폭 30 + paraphrase
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0908_gpt.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

In [ ]:
# 0905 데이터 + 긍부정 비율 2:1 + 에폭 30 + paraphrase
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 왜 맨날 이 가게 들 응 문자 달라 니까 왜 안주세요 다 식어서 먹잖아요 몇번 적어야 됩니까 제 시간 에 가져간 라이더 좀 말좀 하세요 가게 에서 관리 해야 한다고 봅니다
답변    : 소중한 주문 감사 드립니다 찜 해주시 면 정말 감사 드립니다 안녕하세요 소중한 별 과 리뷰 정말 감사합니다 맞습니다 배달 까지 완벽하게 마무리 했어야 하는데 고객 님 께 불편 드린 고개 숙여
예측답변: 저희 매장을 방문해 주신 고객 여러분 감사합니다 다음 번에도 더 좋은 모습으로 찾아 뵙겠습니다 감사합니다
BLEU점수: 0.09980201912515678
리뷰    : 양 맛 다 짱
답변    : 고객 님 감사합니다 맛있게 드신거죠 너무 너무 기뻐요 시국 이 시국 인 만큼 배달 하시는 고객 분들 이 많으신 데 저희 가족 이 먹는다는 마음 으로 최대한 맛있게 드실수있게 나름 연구
예측답변: 골라 먹는 부천대 고객 님 안녕하세요 맛있는 부천대로 찾아 뵙겠습니다. 최고의 맛과 서비스로 보답 하겠습니다.
BLEU점수: 0.12002196056261612
리뷰    : 처음 주문 해봤는데 맛있게 잘 먹었습니다 빨리 와서 좋았어요 다음 에 또 이용 할께요 번창 하세요
답변    : 고객 님 안녕하세요 주 문 주시 고 먹음 직 스런 음식사진 과 더불어 재 주문 해주신다는 말씀 까지 넘 넘 고맙습니다 언제 찾아주셔도 맛있는 음식 으로 고객 님 께 찾아가겠습니다 고객
예측답변: 앞으로도 더 좋은 요리로 찾아 뵐게요 감사합니다
BLEU점수: 0.12673207360719063
리뷰    : 앞 으로도 자주 시킬게요 계속 먹어도 여전히 맛있어요
답변    : 저희 청 주을 찾아주셔서 너무 감사 드려요 이 맛 변치않도록 더 열심히 뛰고 노력 하겠습니다 타코야끼 생각나실 때 마다 저희 잊지말고 찾아주셔요 소중한 시간 내어 리뷰 달아주셔서 감사합니다
예측답변: 항상 고객 만족을 위해 노력 하는 저희가 고객 님 안녕하십니까 고객님의 블로그에 방문해 주시는 것을 너무 감사드립니다.
BLEU점

In [ ]:
# 0905 데이터 + over + 긍부정 비율 2:1 + 에폭 30 + paraphrase
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905_over.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 배달 음식 자주 시켜 먹고 여기 서는 처음 인데 1시간 10분 만에 배달 오고 늦게 온 것 도 그런데 떡 다 불어있고 치즈 뭉쳐서 딱딱하구요 실망 배달 시켜서 기분 안좋긴 처음
답변    : 지금 리뷰 를 봤네요 언제 주문 하신거죠 저희 가 늦게 간다 고 해서 음식 도 오래되는건 극히 드문 일이 에요 늦게가는건 주문 이 밀려서 늦게 만들었기 때문 인데 요 원래 좀
예측답변: 저희 가게를 방문해 주신 고객님께 감사의 뜻을 전하고 싶고 앞으로도 많은 이용 부탁 드리겠습니다.
BLEU점수: 0.09267714327273047
리뷰    : 배달 도 적당한 시간 에 왔는 댕 수 저 젓가락 이 안 왔어요 o 표시 햇 는대 사무실 에서 주 문 한 거 라 너무 당황 했 네 여 담 엔
답변    : 아이고 소중한 고객 님 사죄 의 말씀드립니다 본의 아니게 인도 식 의 식사 로 대접 해드려 버렸네요 다음 엔 절대 절대 네버 빼먹지 않고 잘 챙겨 드리겠습니다 넘 넘 죄송합니다 혹시
예측답변: 저희 음식 은 배달 대행 업체 를 이용 해 요리 과정 에서 부족했던 을 하고 있습니다 일부 용기 에 따라 일부 과정 에 미흡했던 을하고 있습니다
BLEU점수: 0.09100168359620395
리뷰    : 잘 먹었고 맛있게 잘 먹었어요
답변    : 즐거운 여름 휴가 를 떠나시기 전 에 맛있는 음식 으로 건강 도 챙겨 보세요 .
예측답변: 감사합니다. 감사합니다.
BLEU점수: 8.016714023843699e-156
리뷰    : 넘 맛있고 좋았는데 치즈 는 추가 가 된 건지 의문 이 살짝 드네 요
답변    : 맛있게 드셨다니 감사합니다 더 푸 짐하도록 신경 쓰겠습니다 항상 노력 하는 되겠습니다 즐거운 시간 보내시고 행복한 나날 되세요
예측답변: 저희 매장은 더욱 좋은 요리와 서비스로 찾아 뵙겠습니다 고객님 감사합니다.
BLEU점수: 0.09871860383734853
리뷰    : 잘 먹었 습 니당
답변    : 너무 다행 저희 가 준비 한 음식 맛있게 드셔주셔서요 

In [ ]:
# 0905 데이터 + 긍부정 비율 2:1 + 에폭 30 + paraphrase
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 갈비 진짜 너무 맛있네용 반신반의 하면서 시 켯 는데 갈비 맛 나구 진짜 맛 잇 어유 치킨 도 구 웃 잘 먹었 슴 당
답변    : 안녕하세요 맛있게 드셔주셔서 감사합니다 치킨 조합 정말 별미 고객 님 께서 치킨 이 땡기실 때 마다 저희 가 생각나도록 더욱 열심히 하겠습니다 아 주문 정말 감사 드리고 다음 에 또 봐요


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


예측답변: 저희 음식이 입맛에 맞았다면 지금처럼 힘든 시기는 없었을 것이라 생각하는데 고객 여러분의 소중한 후기 덕분입니다 감사합니다.
BLEU점수: 0.2696986267476108
리뷰    : 맛있는 냄새 에 잘 먹을게요 비 조리 인데 아쉽게 리뷰 이벤트 안 적어서 인지 사리 가 없네요
답변    : 에 이 쿵 아쉬워서 어 쩌지 요 말 씀이 없으시면 챙겨 드리 기 가 힘듭니다 다음 에는 꼭 적어 주세요 더 맛있게 드실 수 있게 같이 드릴게요
예측답변: 저희가 이 부분은 운영을 이쪽에서 하기 때문에 고객님들이 불편을 많이 느끼셨겠습니다만 양해 부탁드립니다.
BLEU점수: 3.176496675533262e-78
리뷰    : 옆집 문 앞 에 배달 해주셨네요 맛있어서 다 먹고 사짘 찍어 버렸어요 잘 먹었습니다
답변    : 앗 죄송합니다 고객 님 그래도 너그러이 이해 해주시고 리뷰 까지 남겨주셔서 너무 감사해요 더 노력 하는 되겠습니다 늘 행복하세용


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


예측답변: 안녕하세요 고객 님 다음에도 더 좋은 서비스와 맛있는 요리로 찾아 뵙겠습니다.
BLEU점수: 0.24830724520513484
리뷰    : 몸 이 안좋아서 시켜 봤는데 양도 많고 맛 도 좋아요
답변    : 리뷰 남겨주셔서 감사합니다 칭찬 해 주시 니 힘 이 팍팍 나네요 저희 음식 이 기력 을 회복하시는데 도움 이 되기만 한다면 그것 만으로도 너무나 행복합니다 발전 하는 맛 과 좋은 서비스
예측답변: 고객 여러분 감사합니다 이렇게 좋은 요리 만들어 주셔서 요즘 같이 힘든 시기에 정말 감사드립니다.
BLEU점수: 0.22491637465159936
리뷰    : 먹고나면 속이 편하고 확실히 맛있어요 가라 아케 도 오랜 만 에 시켜 먹었는데 잘 먹었습니다
답변    : 반갑습니다 오늘 도 맛있게 드시고 예쁜 사진 리뷰 정말 감사 드립니다
예측답변: 고객 님 맛있게 먹어주셔서 감사합니다.
BLEU점수: 0.19154624426653655
리뷰    : 맛 도 좋고 배달 도 빨 라서 따뜻하게 맛있게 먹었어요
답변    : 즐거운 시간 보내시고 소중한 리뷰 작성 해주셔서 감사 드려요 메뉴 별로 가장 맛있는 온도 가 있다고 해 요 많은 분 들 이 매장 에서 찾아주시는 이유 이기도 하죠 저희 도 이러한
예측답변: 당일 삶은 보쌈 직접 담근 양념과 신선한 음식 재료로 고객 밥상에 항상 오르는 것이 바로 이 요리입니다 맛있게 먹어 주셔서 감사합니다
BLEU점수: 0.07196816797546395
리뷰    : 폴트 버거 너무 맛있고 감자 튀김 도 정말 맛있네요 배달 도 빨리 해주셔서 따뜩하 게 잘 먹었어요 포장 도 너무 예쁘고 테니스 컨셉 매장 도 꼭 한번 가보고싶어요
답변    : 소중한 리뷰 글 감사 드립니다 저희 항상 고객 님 을 향 한 마음 으로 맛있는 수제 찾아 뵙도록 노력 하겠습니다 감사합니다
예측답변: 당일 삶은 삼겹살과 직접 담근 김치, 그리고 신선한 식자재를 가지고 고객 밥상에 매번 정성을 다하는 요리 연구가 계속 되고 있습니다.
BLEU점수:

In [ ]:
# 0905 데이터 + 긍부정 비율 5:5 + 에폭 30
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_0905.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 간장 은 묽어서 좀 아쉬운데 나머진 맛 잇어요
답변    : 고객 님 안녕하세요 골라 먹는 부천대 주문 에 맛있게 드셔 주셔서 정말 감사합니다 맛있게 드셨다는 칭찬 은 언제 들어도 참 기분 이 좋아지는 그런 말 인거 같아요 덕분 에 힘든 시기
예측답변: 고객 님 소중한 리뷰 감사합니다 저희는 저희 민의의 고기를 두 툼 하 게 썰고 있습니다 광어 도미 등 이 이 이 저희 매장으로 연락 주세요 묻고 따지지
BLEU점수: 0.1506279696450486
리뷰    : 오 여기 좋네요 시켜놓고 낮잠 자 다가 늦게 먹었긴 했는데 진짜 빨리 와요
답변    : 감사합니다 주문 즉시 최대한 빨리 조리 해서 보내 드리고있어요 빠른 베차 시 스팀 이기 에 고객 님 들이 우선 적 으로 잡아 가는 편이 에요
예측답변: 안녕하세요 고객 님 소중한 리뷰 감사합니다 주셔서 감사합니다 감사드립니다 항상 고객 님의 소리는 저희를 춤추게 합니다 앞으로 계속해서 정성스러운 음식을 제공해 드리도록 약속
BLEU점수: 0.18665030216111525
리뷰    : 닭발 좋아해서 여기저기 주문 해서 먹는데 공주 닭발 맛있습니다 배달 도 무지 빠르고 센스 있는 서비스 까지 감사합니다 맛있게 잘 먹었습니다
답변    : 항상 같은 맛 유지 하도록 노력 할게요 항상 맛 과 청결 에 정성 을 다 하겠습니다 주문 해주셔서 감사합니다
예측답변: 고객 님 소중한 리뷰 감사합니다 감사드립니다 고객 님의 만족을 위해 노력하는 되겠습니다 즐거운 시간 보내시고 행복한 나날 되세요
BLEU점수: 0.18289441873227932
리뷰    : 최고 너무 맛있고 배달 도 빠르고 잘 먹었습니다
답변    : 반갑습니다 맛있게 드셔주시고 잊지 않고 리뷰 남겨 주심 에 큰 힘 을 얻습니다 늘 변함 없는 맛 과 친절한 서비스 로 보답 하겠습니다 항상 행복하세요
예측답변: 안녕하세요 고객 님 저희 매장으로 찾아뵙겠습니다
BLEU점수: 0.0218228208440802
리뷰    : 오늘 은 주 문 한 거 빠

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


예측답변: 주문해 주셔서 감사합니다 드립니다 다음에 또 주문해 주세요
BLEU점수: 0.06122493138554343
리뷰    : 전체 적 으로 맛있네요 하와 이안 에 코코넛 은 좀 그렇긴 한데 요 기 서 만 계속 피자 먹네요
답변    : 맛있게 드셔주시고 소중한 리뷰 달아주셔서 정말 감사 드립니다
예측답변: 안녕하세요 고객 님 저희 매장을 찾아주셔서 너무 감사드립니다 앞으로도 저희 음식을 이용해 주셔서 정말 감사드립니다 다음에 또 주문해 주세요
BLEU점수: 0.18463832893957804
리뷰    : 사장 님 맛있게 잘 먹었습니다 번창 하세요
답변    : 안녕하세요 고객 님 맛있게 드시고 소중한 리뷰 와 별 5 개 감사합니다 언제 주문 하셔도 푸 짐 하고 맛있는 닭발 을 위해 항상 최선 을 다 하겠습니다 앞 으로도 많은 사랑
예측답변: 안녕하세요 고객 님 맛있게 드시고 소중한 리뷰 남겨주셔서 정말 감사합니다 드립니다 고객 님의 소리는 저희를 춤추게 합니다 앞으로 계속해서 정성스러운 음식을 제공해 드리도록 약속
BLEU점수: 0.4093854533293307


In [ ]:
# yogiyo 전체데이터 모델링
STATEDICT_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_paraphrase.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 먹을만 합니다 추천 해 요
답변    : 좋은 리뷰 감사 드립니다 저희 은 리뷰 평점 이나 표면 적 으로만 좋은 집 이 아닌 드셧 던 분들 이 믿고 다시 드시고싶은 집 이 되자 라는 마음 으로 모두 가
예측답변: 고객 님 소중한 고객 네 이 이임 이임 맛있게 드셨다니 너 어어 어 무 기쁜 네 용 만족스러운 식사하셨다면에 용 만족스러운 식사하셨다면 찜 가
BLEU점수: 1.7456357302038282e-78
리뷰    : 여친 이 너무 맛있게 잘 먹었답니다 감사합니다 사장 님 번창 하세요
답변    : 안녕하세요 토핑 을 아끼지 않는 미 미 주문 주시 고 리뷰 로 찾아와 주셔서 정말 감사 드립니다 앞 으로도 맛있는 생각나실 때 저희 미 또 찾아주세요 감사합니다
예측답변: 맛있게 드셔주셔서 감사합니다 요청 사항 은 조 리전 늘 꼼꼼하게 체크하고 있으니 필요하신 사항 있으시면 다음 주 문 시 주 문 메모에 남겨주세요 최대한 반영해서 보내 드리겠습니다 믿고 주문해
BLEU점수: 0.1556563403952443
리뷰    : 주말 의 마무리 는 역시
답변    : 고객 님 리뷰 에 몸 이 절로 들썩입니다 시간 날 때 마다 리뷰 훑어 보며 반응 살피 고 있는데 덕분 에 넘 힘 이나고 힐링 이 되네요 리뷰 너무 감사 드립니다
예측답변: 안녕하세요 고객 님 소중한 리뷰 남겨주셔서 감사합니다 저희 큰아들 백 순대는 항상 맛있는 서비스를 해드리기 위해 서 노력하고 있습니다 사용되는 모든 곱창 은 국내 산 제 주산
BLEU점수: 0.12288378213016357
리뷰    : 고기 가 맛있어요 사장 님 이 서비스 로 쌈장 도 주셨어요 배달 도 예상 시간 보다 빠흐 게 옵니다
답변    : 저희 목살 만족스럽고 맛있게 잘 드셨다니 기쁩니다 소중한 리뷰 와 저희 에게 큰 힘 을 주는 좋은 평가 남겨 주셔서 정말 감사 드립니다 무더운 날씨 에 건강 잘 지키시고 오늘 도
예측답변: 맛있게 드셔 주시 고 소중한 리뷰 남겨주셔서 감사합니다 만족스러운 식사하신 것 같아 오

In [ ]:
# yogiyo 전체데이터 모델링
STATEDICT_OV_PATH = '/content/drive/MyDrive/project_final/data/models/yogiyo_paraphrase_clean.pt'
model.load_state_dict(torch.load(STATEDICT_OV_PATH))
random_evaluation_10(model, test_dataset, train_dataset.wordvocab.index2word, device)

리뷰    : 주문 에 후라이드 잇 슈가 있었지만 다 맛있었습니다
답변    : 저희 을 항상 찾아주시고 성원 해주시는 분 들 께 감사 의 뜻 을 전하 고 싶을 만큼 귀한 시간 내어 주셔서 감사합니다 .
예측답변: 항상 저희 직구 삼이 직구 삼이 되도록 노력 직구 삼이 되도록 노력 하겠습니다 감사합니다 .
BLEU점수: 0.14828218247800706
리뷰    : 맛있습니다 배달 도 빠르고 양도 많아오
답변    : 항상 건강하고 행복한 나날 들 이 되시길 기원 합니다 . 고객 님 들의 소중한 정보 감사 드립니다 .
예측답변: 항상 건강하고 행복한 나날 들 이 되시길 기원 합니다 .
BLEU점수: 0.4464393987758162
리뷰    : 디저트 하나 하나 다 맛있어요 휘 낭시 에 랑 소금 빵 완전 맛있어요 이 즈 니 버터 프레첼 은 팥 앙금 들어있는건 줄 알았는데 다시 보니 아니더라 규 요 그치만 이
답변    : 이 즈 니 버터 프리첼 의 팥 앙금 버전 앙 버터 도 뚜레쥬르 팀 에 제안 해 보겠습니다 .
예측답변: 이 즈 니 버터 프리첼 의 팥 앙금 버전 앙 버터 도 뚜레쥬르 팀 에 제안 해 보겠습니다 .
BLEU점수: 1.0
리뷰    : 국물 이 깔끔하고 맛있어요 또 먹고 싶은 맛 이네 요
답변    : 저희 국밥 을 많이 사랑 해 주세요 :) 항상 최선 을 다해 준비 하는 만큼 더 좋은 맛 과 영양 으로 보답 하겠습니다 .
예측답변: 저희 국밥 을 많이 사랑 해 주세요 :) 항상 최선 을 다해 준비 하는 만큼 더 좋은 맛 과 영양 으로 보답 하겠습니다 .
BLEU점수: 1.0
리뷰    : 커피 도 부드럽고 다 맛 나네요 잘 먹었습니다
답변    : 항상 정갈한 브런치 맛있는 브런치 준비 하고 고객 님 기다릴게요 곧 또 만나요 그때 까지 행복한 시간 되세요 .
예측답변: 항상 정갈한 브런치 는 브런치 준비 가 준비 기다릴게요 기다릴게요 곧 또 찾아주세요 행복한 하루 되세요 .
BLEU점수: 0.5833316665807692
리뷰    : 맛